In [50]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = Docx2txtLoader("./tax_with_markdown.docx")
# document = loader.load()

# print(document)
# len(document)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,  # 문서 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap = 200,  # 쪼갤 때, 별개로 짜르는 게 아니라, 겹치게 해서 split 
)

document_list = loader.load_and_split(text_splitter=text_splitter)
# print(document_list)

In [4]:
import os 

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

load_dotenv() # 환경변수 불러오기 

embedding = OpenAIEmbeddings(model='text-embedding-3-large') 
index_name = 'tax-markdown-index'
database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)


query = '연봉 5천만원인 직장인의 종합소득세는 얼마인가요?'


llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull("rlm/rag-prompt")
retriever = database.as_retriever(search_kwargs={'k': 7})

# QA chain 만들기 
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever, 
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

dictionary = ["사람을 나타내는 표현 -> 거주자"]
prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다. 
    그런 경우에는 {{question}}
    사전: {dictionary}
    질문: {{question}}
""")

# query -> 직장인 -> 거주자 chain 추가 
dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query": dictionary_chain} | qa_chain

ai_response = tax_chain.invoke({"question": query})

print(ai_response["result"])

d:\Program\miniconda3\envs\study\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


연봉 5천만원인 거주자의 종합소득세는 624만원입니다. 이는 5,000만원 초과 금액에 대해 추가 세율이 적용되지 않기 때문입니다.
